In [1]:
# Import / install relevant Python packages
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
window = 48
# I will continue with 2 ( since in paper it is used average for 1990 - 2020 - 3157 )
# and with using 2 I get 3284 ( for 1 is 2878)
max_number_of_nulls_in_observation = 2

In [3]:
feature_names_list = ['mom1m','absacc','acc','aeavol','age','agr','baspread',
'beta','betasq','bm','bm_ia','cash','cashdebt','cashpr','cfp','cfp_ia',
'chatoia','chcsho','chempia','chinv','chmom','chpmia','chtx','cinvest',
'convind','currat','depr','divi','divo','dolvol','dy','ear','egr','ep',
'gma','herf','hire','idiovol','ill','indmom','invest','lev','lgr','maxret',
'ms','mve_ia','mvel1','nincr','operprof','pchcapx_ia','pchcurrat','pchdepr',
'pchgm_pchsale','pchquick','pchsale_pchrect','pctacc','pricedelay',
'ps','quick','rd','retvol','roaq','roeq','roic','rsup','salecash',
'salerec','securedind','sgr','sin','sp','std_dolvol','std_turn',
'tang','tb','turn','zerotrade']

In [4]:
mom_names_list = ["mom1m","mom2m","mom3m","mom4m","mom5m","mom6m","mom7m","mom8m","mom9m","mom10m","mom11m","mom12m",
                "mom13m","mom14m","mom15m","mom16m","mom17m","mom18m","mom19m","mom20m","mom21m","mom22m","mom23m","mom24m",
                "mom25m","mom26m","mom27m","mom28m","mom29m","mom30m","mom31m","mom32m","mom33m","mom34m","mom35m","mom36m",
                "mom37m","mom38m","mom39m","mom40m","mom41m","mom42m","mom43m","mom44m","mom45m","mom46m","mom47m","mom48m"]


In [5]:

df_radni = pd.read_csv('D:/USB Drive/Data/DachengXiu/df_radni.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'D:/USB Drive/Data/DachengXiu/df_radni.csv'

In [ ]:
#how many nuuls per parameter - in dataset
temp = df_radni.isnull().sum(axis = 1).sort_values()
temp[temp > 0].plot.hist(bins=81, alpha=0.5)

In [ ]:
#how many nuuls per parameter - after 1990
temp = df_radni[(df_radni['DATE'] >= 19891231 )].isnull().sum(axis = 1).sort_values()
temp[temp > 0].plot.hist(bins=81, alpha=0.5)

In [ ]:
def how_many_companies(df_window):
    #Last date
    last_date = sorted(df_window['DATE'].unique())[-1]
    #unique companies in window
    unique_companies = df_window['permno'].unique()
    list_of_companies_with_null_mom1m = []
    for x in unique_companies:
        #df for for all observatins for one company - should be 48 - if it less all shoud be droped
        df_window_for_company = df_window[df_window['permno'] == x]
        #length od window should be 48
        length_of_window = len(df_window_for_company)
        if (length_of_window == window):
            #checking number of Null in mom1m in observations - max 48
            number_of_null = df_window_for_company['mom1m'].isnull().sum()
            if (number_of_null == 0):
                #umber of nulls - we check if there is more then max_number_of_nulls_in_observation of features on last date
                number_of_nulls_features = df_window_for_company[df_window_for_company["DATE"] == last_date].isnull().sum(axis = 1).mean()
                if not (number_of_nulls_features <= max_number_of_nulls_in_observation):
                    list_of_companies_with_null_mom1m.append(x)
            else:
                list_of_companies_with_null_mom1m.append(x)
        else:
            list_of_companies_with_null_mom1m.append(x)
    #drop observations of companies with more then max allowed number of null-s in features or withh less then 48 observations or with null in 48 obs of mom1
    df_window = df_window[~df_window.permno.isin(list_of_companies_with_null_mom1m)]
    return len(df_window['permno'].unique())

In [ ]:
#checking number of companies after 1990
df_radni_small = df_radni[(df_radni['DATE'] > 19851231) & (df_radni['DATE'] < 20210101)]

In [ ]:
#go through list of dates
results = []
dates = sorted(df_radni_small['DATE'].unique())
for date in range(window-1,len(dates)):
    print( "dates[i - window]" , dates[date - window+1]," dates[i] " , dates[date])
    df_window_working = df_radni_small[(df_radni_small['DATE'] >= dates[date - window+1] ) & (df_radni_small['DATE']  <= dates[date])]
    results.append(how_many_companies(df_window_working))


In [ ]:
from datetime import date


def int2date(argdate: int) -> date:
    """
    If you have date as an integer, use this method to obtain a datetime.date object.

    Parameters
    ----------
    argdate : int
      Date as a regular integer value (example: 20160618)

    Returns
    -------
    dateandtime.date
      A date object which corresponds to the given value `argdate`.
    """

    dates = []

    for i in argdate:
      year = int(i / 10000)
      month = int((i % 10000) / 100)
      day = int(i % 100)
      dates.append(date(year, month, day))

    return dates

In [ ]:
df_results = pd.DataFrame(np.column_stack([dates[window-1:(window + len(results))], results]),
                               columns=['Dates', 'Number_of_companies'])
df_results_dates = pd.DataFrame(np.column_stack([int2date(dates[(window-1):(window + len(results))]), results]),
                               columns=['Dates', 'Number_of_companies'])


In [ ]:
df_results_dates.plot(x="Dates", y="Number_of_companies")

We can see that every year on 08.31. there are more missing values then other months.

In [ ]:
df_results[(df_results['Dates']) > 19891231 & (df_results['Dates'] < 20210101)]['Number_of_companies'].mean()

In [ ]:
def clean_df_of_companies_with_non_adequate_observations(df_for_clean):
    counter_less_48 = 0
    counter_some_mom1m_null = 0
    counter_more_nulls_in_features_than_alowed = 0
    #Last date
    last_date = sorted(df_for_clean['DATE'].unique())[-1]
    #unique companies in window
    unique_companies = df_for_clean['permno'].unique()
    list_of_companies_with_null_mom1m = []
    for x in unique_companies:
        #df for for all observatins for one company - should be 48 - if it less all shoud be droped
        df_for_clean_for_company = df_for_clean[df_for_clean['permno'] == x]
        #length od window should be 48
        length_of_window = len(df_for_clean_for_company)
        if (length_of_window == window):
            #checking number of Null in mom1ms in observations - max 48
            number_of_null = df_for_clean_for_company['mom1m'].isnull().sum()
            if (number_of_null == 0):
                #umber of nulls - we check if there is more then max_number_of_nulls_in_observation of features on last date
                number_of_nulls_features = df_for_clean_for_company[df_for_clean_for_company["DATE"] == last_date].isnull().sum(axis = 1).mean()
                if not (number_of_nulls_features <= max_number_of_nulls_in_observation):
                    list_of_companies_with_null_mom1m.append(x)
                    counter_more_nulls_in_features_than_alowed = counter_more_nulls_in_features_than_alowed +1
            else:
                list_of_companies_with_null_mom1m.append(x)
                counter_some_mom1m_null = counter_some_mom1m_null +1
        else:
            list_of_companies_with_null_mom1m.append(x)
            counter_less_48 = counter_less_48 +1
    #drop observations of companies with more then max allowed number of null-s in features or withh less then 48 observations or with null in 48 obs of mom1
    df_for_clean = df_for_clean[~df_for_clean.permno.isin(list_of_companies_with_null_mom1m)]
    print("Length od window less then 48 ", counter_less_48," Number of Null in mom1ms " ,counter_some_mom1m_null , "Number of nulls in features more than alowed ", counter_more_nulls_in_features_than_alowed )
    return df_for_clean

In [ ]:
df_moms = pd.DataFrame(columns=mom_names_list[1:])
df_empty_copy_of_radni = pd.DataFrame(data=None, columns=df_radni.columns, index=df_radni.index)
df_with_moms = pd.concat([df_empty_copy_of_radni, df_moms],axis=1 ).iloc[0:0]


In [ ]:
def add_moms_features_for_last_date_and_drop_observations_of_other_dates(df_for_adding_moms):
    #adding empty moms columns
    df_for_adding_moms = pd.concat([df_for_adding_moms, df_moms],axis=1 )
    #Last date
    last_date = sorted(df_for_adding_moms['DATE'].unique())[-1]
    #go through list of companies
    for company in df_for_adding_moms['permno'].unique():
        #taking mom1m values from 48 rows
        moms_list = df_for_adding_moms[df_for_adding_moms['permno'] == company].sort_values('DATE')
        #detrming index of last date row
        indeks = moms_list[moms_list['DATE']== last_date].index[0]
        #making list of reverse order since it is from older date to newer
        moms_list = moms_list['mom1m'].iloc[::-1].values.tolist()
        #calculate moms from formula mom1m, mom2m = mom1(shifted 1), ... , momim = ( momi-1m + 1)* ( mom1(shifted i-1) + 1) -1
        for i in range(2,len(moms_list)):
            moms_list[i] = (moms_list[i-1]+1)*(moms_list[i]+1)-1
        # fill values of moms ( from 48 rows ) to columns
        df_for_adding_moms.loc[indeks,mom_names_list] = moms_list
    #drop all exept last date
    df_for_adding_moms = df_for_adding_moms[df_for_adding_moms['DATE'] == last_date]
    return df_for_adding_moms

In [ ]:
#go through list of dates
results = []
dates = sorted(df_radni['DATE'].unique())
for date in range(window-1,len(dates)):
    print( "dates[i - window]" , dates[date - window+1]," dates[i] " , dates[date])
    df_window_working = df_radni[(df_radni['DATE'] >= dates[date - window +1] ) & (df_radni['DATE']  <= dates[date])]
    df_window_cleaned = clean_df_of_companies_with_non_adequate_observations(df_window_working)
    df_window_with_moms = add_moms_features_for_last_date_and_drop_observations_of_other_dates(df_window_cleaned)
    #results.append(how_many_companies(df_window_working))
    df_with_moms = pd.concat([df_with_moms, df_window_with_moms],axis=0 )


In [ ]:
df_with_moms.to_csv('D:/USB Drive/Data/DachengXiu/df_with_moms.csv')

In [ ]:
df_with_moms = pd.read_csv('D:/USB Drive/Data/DachengXiu/df_with_moms.csv')

In [ ]:
#how many nulls per parameter - in dataset
temp = df_with_moms.isnull().sum(axis = 1).sort_values()
temp[temp >= 0].plot.hist(bins=81, alpha=0.5)

It is small number in entire sample but is more important by date.

In [ ]:
#determing number of nulls by features
count_nuns_by_feature=df_with_moms.isnull().astype(int).sum().sort_values(ascending = False)
column_names_with_null_values_features_list = count_nuns_by_feature[count_nuns_by_feature > 0].sort_values(ascending = True).keys().values.tolist()

In [ ]:
# Create a bar chart to visualize count of nulls by features
plt.figure(figsize=(16, 96))
count_nuns_by_feature[column_names_with_null_values_features_list].plot.barh()
plt.show()

In [ ]:
count_nuns_by_feature

In [ ]:
#calculate nuns in features by date
column_names_with_null_values_features_list.append("DATE")
df_nuuls_of_features_by_date = df_with_moms[column_names_with_null_values_features_list].copy()
df_nuuls_of_features_by_date.set_index('DATE')
column_names_with_null_values_features_list.remove("DATE")
df_nuuls_of_features_by_date[column_names_with_null_values_features_list]=df_nuuls_of_features_by_date[column_names_with_null_values_features_list].isnull().astype(int)
df_nuuls_of_features_by_date = df_nuuls_of_features_by_date.groupby('DATE').sum()
#df_nuuls_of_features_by_date = df_nuuls_of_features_by_date.groupby('DATE').sum().sort_values('tb',ascending=False)

In [ ]:
#max missings values in features by single date
df_nuuls_of_features_by_date.max().sort_values(ascending=False).head(10)

We can see that:
- tb has most missing values. This 535 was on 20051031 and tb was only missing value. tb is not bigger than 16%
- chtx 324 on 19830729.Few months before and after there were some values missing but after there is no missing values for chtx
- bm,bm_ia las few dates are miising 20211231
- aeavol is few months oround 19830531
- operprof is more frequent but is not bigger than 6%

In general tb and operprof are more frequent. Others are just pop up for few months on  the level lower than tb.

In [ ]:
#percentage of tb and operprof missing values by date
df_tb_operprof_by_date = pd.merge(df_nuuls_of_features_by_date[['tb','operprof']],df_with_moms.groupby('DATE').count()['permno'], on=["DATE","DATE"])
df_tb_operprof_by_date["tb %"] = df_tb_operprof_by_date['tb']/df_tb_operprof_by_date['permno']*100
df_tb_operprof_by_date["operprof %"] = df_tb_operprof_by_date['operprof']/df_tb_operprof_by_date['permno']*100
df_tb_operprof_by_date = df_tb_operprof_by_date.drop('tb',axis=1)
df_tb_operprof_by_date = df_tb_operprof_by_date.drop('operprof',axis=1)
df_tb_operprof_by_date = df_tb_operprof_by_date.drop('permno',axis=1)
df_tb_operprof_by_date.plot()

If we inpute median values, it is dificult to estimate if max 16% imputed values, in one of out of 76 ( 76+48=124 ) features would make spurious conclusions.
All analysis should be taken for max_number_of_nulls_in_observation = 1 and max_number_of_nulls_in_observation = 0 ( we are using 2 since it give us same average number of companies as it is stated in paper)

In [ ]:
#go through list of dates and fill NA with medion for that date
dates_for_imputation = sorted(df_with_moms['DATE'].unique())
for date in dates_for_imputation:
    df_with_moms[df_with_moms['DATE'] == date] = df_with_moms[df_with_moms['DATE'] == date].mask(df_with_moms[df_with_moms['DATE'] == date].isna(), other=df_with_moms[df_with_moms['DATE'] == date].median(), axis=1)

In [ ]:
df_describe=df_with_moms[feature_names_list].describe(percentiles=[0.01,0.05, 0.5,0.95, 0.99])

In [ ]:
(df_describe.T['max']/df_describe.T['95%']).sort_values(ascending=False).head(25)

In [ ]:
df_describe

In [ ]:
# EDA: Plot histograms for all variables (features + target)
df_with_moms[feature_names_list].hist(figsize=(36, 24), bins=50, edgecolor='black')
plt.subplots_adjust(hspace=0.5, wspace=0.5)
plt.show()

We are looking for similar companies, so by winzdorize or some other treatment of outliers could make more demage than benefit?

In [ ]:
#df_with_moms.to_csv('D:/USB Drive/Data/DachengXiu/df_preprocessing_done.csv')